<a href="https://colab.research.google.com/github/Aryu-Tamura/GoogleColab_ChatbotTest/blob/main/04_%E8%8B%B1%E4%BC%9A%E8%A9%B1%E3%83%81%E3%83%A3%E3%83%83%E3%83%88%E3%83%9C%E3%83%83%E3%83%88_MVP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**MVP4: UI改善・画像・履歴表示**

目的: 音声入出力、会話履歴（テキスト表示）、画像表示機能を持つ、シンプルで理解しやすいWebアプリを作成します。

手順:

1. 必要なライブラリをインストールします。
2. ライブラリを読み込みます。
3. OpenAI APIキーを設定します。
4. 表示したい画像をアップロードします。
5. 会話履歴を保存する変数 (`history`) を準備します。
6. シンプルな音声認識関数を作成します。
7. シンプルな音声合成関数を作成します。
8. 新しいシンプルなメイン関数を作成します (履歴テキスト整形含む)。
9. Gradioを使って、レイアウトを整え、画像と履歴(テキスト)を表示するUIを作ります (Blocks を使用)。
10. Webアプリを起動します。

-------------------------------

1. ライブラリのインストール



In [ ]:
!pip install openai
!pip install gradio

-------------------

2. ライブラリのインポート



In [ ]:
import gradio as gr
import openai
from google.colab import userdata
import os
import datetime

-----------------------


3. OpenAI APIキーの設定

※事前にColabの左メニュー(🔑)で `OPENAI_API_KEY` という名前でAPIキーを設定しておいてください。

In [ ]:
from google.colab import userdata # APIキーを安全に読み込むため
key = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=key)

-------------------------

4. 表示する画像の準備

In [ ]:
# Colabに画像ファイルをアップロードして変数に読み込む
from google.colab import files

print("表示する画像を1枚アップロードしてください...")
uploaded = files.upload()

image_to_display = None # 初期値

# アップロードされたファイル名とサイズを表示
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

# 最初のファイル名を取得し、Gradioで使えるファイルパスを作成
filename = list(uploaded.keys())[0]
image_to_display = os.path.abspath(filename) # 絶対パスを取得

--------------------

5. 会話履歴 (history) の準備

※このセルはノートブックを開いて最初に一度だけ実行します。




In [ ]:
# グローバル変数として会話履歴を初期化
history = []

# AIへの指示（システムプロンプト）を定義
system_prompt = """あなたは、ユーザーと自然で流暢な英会話を行うAI英語教師です。

# あなたの役割
- ユーザーの英語力向上のために、自然で日常的な英会話を行ってください。
- ユーザーの英語表現や文法に間違いがあれば、会話の流れを止めないように、自然な形で指摘し、より適切な表現を提案してください。
- ユーザーの質問や発言には、共感的に、そして前向きな態度で応答してください。

# 回答の形式
- 会話は常に英語で行ってください。日本語で応答してはいけません。
- 回答は、原則として短く（1～3文程度）、自然な口語表現を使ってください。
- ユーザーが文法や表現のミスをした場合は、以下のような形式で訂正例を示してください。
  例 (Example): User: "I goed to park yesterday." AI: "Oh, nice! You mean, 'I went to the park yesterday,' right?"

# 入出力の例 (Input/Output Examples)
User: "Hello! How was your day?" AI: "Hey! My day's going great, thanks for asking. How about you?"
User: "I'm study English today." AI: "That's great! Just a quick tip: you'd say, 'I'm studying English today.' What are you working on right now?"
User: "Can you help me about grammar?" AI: "Of course! I'd say, 'Can you help me with grammar?' Sure thing! What grammar point do you want to talk about?" """

# historyリストの最初にシステムプロンプトを追加
history.append({"role": "system", "content": system_prompt})

print("会話履歴(history)を初期化しました。")

---------------------------


6. シンプルな音声認識関数



In [ ]:
def speech_to_text_simple(audio_filepath):
    """簡単な音声認識関数 (Whisper)"""
    if audio_filepath is None: return ""
    try:
        audio_file = open(audio_filepath, "rb")
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            language="en")
        return transcription.text
    except Exception as e:
        print(f"音声認識エラー: {e}")
        return ""

-----------------------------


7. シンプルな音声合成関数



In [ ]:
def text_to_speech_simple(text, output_dir="tts_output_simple_v4"): # 保存先フォルダ名変更
    """簡単な音声合成関数 (OpenAI TTS)"""
    if not text: return None
    try:
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        speech_file_path = os.path.join(output_dir, f"response_{timestamp}.mp3")
        response = client.audio.speech.create(model="tts-1", voice="alloy", input=text)
        response.stream_to_file(speech_file_path)
        return speech_file_path
    except Exception as e:
        print(f"音声合成エラー: {e}")
        return None

-------------------------

8. シンプルなメイン関数

In [ ]:
def english_teacher_bot_final(input_mode, input_text, input_audio):
    """
    入力に応じてAIと対話し、履歴を更新し、
    入力テキスト、応答テキスト、応答音声パス、履歴テキストを返す関数。
    """
    global history

    # 1. 入力モードに応じてテキスト化 (簡単なフォールバック付き)
    processed_input_text = ""
    if input_mode == "Text":
        processed_input_text = input_text
    elif input_mode == "Voice":
        if input_audio is not None:
            try:
                processed_input_text = speech_to_text_simple(input_audio)
            except Exception as e: # 音声認識中の予期せぬエラー
                 print(f"音声認識中に予期せぬエラー: {e}")
                 processed_input_text = "" # エラー時は空にする
            if not processed_input_text: # 認識失敗またはエラーの場合
                print("音声認識失敗。テキスト入力があれば使います。")
                processed_input_text = input_text # テキスト入力でフォールバック
        else: # Voiceモードで音声がない場合
             print("音声モードですが音声がありません。テキスト入力があれば使います。")
             processed_input_text = input_text # テキスト入力でフォールバック

    print(f"処理された入力テキスト: {processed_input_text}")

    # 有効な入力がない場合は終了
    if not processed_input_text:
        # 戻り値の数を合わせる (履歴テキストは現在の履歴から生成)
        history_display_text = "--- 会話履歴 ---\n"
        for msg in history:
             if msg["role"] == "user": history_display_text += f"あなた: {msg['content']}\n"
             elif msg["role"] == "assistant": history_display_text += f"AI: {msg['content']}\n\n"
        return "(入力なし)", "(入力がなかったため応答できません)", None, history_display_text

    # 2. ユーザー入力を履歴に追加
    history.append({"role": "user", "content": processed_input_text})

    # 3. ChatGPT API呼び出し
    assistant_response_text = ""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo", messages=history, max_tokens=100, temperature=0.7
        )
        assistant_response_text = response.choices[0].message.content
    except Exception as e:
        assistant_response_text = f"エラー: {e}"
        print(assistant_response_text)
        history.pop() # エラー時はユーザー入力削除

    # 4. AI応答を履歴に追加
    history.append({"role": "assistant", "content": assistant_response_text})
    print(f"AI応答テキスト: {assistant_response_text}")

    # 5. 音声合成
    response_audio_path = None
    if not assistant_response_text.startswith("エラー"):
        response_audio_path = text_to_speech_simple(assistant_response_text)

    # 6. 履歴表示用テキストを作成 (毎回全履歴を整形)
    history_display_text = "--- 会話履歴 ---\n"
    # システムプロンプトを除いて表示する場合 (オプション)
    # for msg in history[1:]: # インデックス1から開始
    for msg in history: # システムプロンプトも含めて表示する場合
        prefix = ""
        if msg["role"] == "system": prefix = "システム設定:\n"
        elif msg["role"] == "user": prefix = f"あなた ({input_mode}): " # 入力モードも表示
        elif msg["role"] == "assistant": prefix = "AI: "
        history_display_text += f"{prefix}{msg['content']}\n\n" # メッセージ間に空行

    # 7. 結果を返す (入力テキスト, 応答テキスト, 応答音声パス, 履歴テキスト)
    return processed_input_text, assistant_response_text, response_audio_path, history_display_text

print("メイン関数 'english_teacher_bot_final' を定義しました。")

--------------------

9. Gradio UI 作成 (Blocks、画像、履歴テキスト表示)



In [ ]:
# gr.Blocks を使ってUIを作成します
with gr.Blocks() as app:
    gr.Markdown("# 英会話チャットボット") # タイトル統一
    gr.Markdown("テキストまたは音声で入力し、「送信」ボタンを押してください。")

    # UIを左右2列に分割
    with gr.Row():
        # 左列: 入力と履歴表示
        with gr.Column(scale=2): # 左を広めに
            gr.Markdown("### 入力")
            input_mode = gr.Radio(["Text", "Voice"], label="入力方法", value="Text")
            input_text = gr.Textbox(label="テキスト入力", placeholder="または、下で録音")
            input_audio = gr.Audio(sources=["microphone"], type="filepath", label="音声入力")
            submit_button = gr.Button("送信", variant="primary")
            gr.Markdown("### 会話履歴")
            # 履歴表示用の複数行テキストボックス (読み取り専用にしない場合は interactive=True)
            history_display = gr.Textbox(label="会話ログ", lines=15, interactive=False)

        # 右列: 出力と画像
        with gr.Column(scale=1):
            gr.Markdown("### 応答 / 結果")
            # 戻り値1 (processed_input_text) を表示 (音声認識結果の確認用)
            recognized_text = gr.Textbox(label="入力内容", interactive=False)
            # 戻り値2 (assistant_response_text) を表示
            output_text = gr.Textbox(label="AIの応答 (テキスト)", interactive=False, lines=4)
            # 戻り値3 (response_audio_path) を表示
            output_audio = gr.Audio(label="AIの応答 (音声)", type="filepath", interactive=False)
            gr.Markdown("### 画像")
            # 画像表示
            display_image = gr.Image(value=image_to_display, label="表示画像", interactive=False)

    # --- イベントリスナーの設定 ---
    submit_button.click(
        fn=english_teacher_bot_final, # 作成したメイン関数
        # 入力: モード, テキスト, 音声 (履歴はグローバル変数)
        inputs=[input_mode, input_text, input_audio],
        # 出力: 認識テキスト, 応答テキスト, 応答音声, **履歴表示テキスト**
        outputs=[recognized_text, output_text, output_audio, history_display]
    )
    # テキスト入力時のEnterキー送信も同様に設定 (オプション)
    # input_text.submit(...)

print("Gradioインターフェースを作成しました。")

--------------------

10. Webアプリの起動



In [ ]:
# 作成したUIを起動します
app.launch(share=True)